In [1]:
from music21 import converter, instrument, note, chord
import glob
import numpy as np
import pickle

In [2]:
notes = []

In [3]:
for file in glob.glob(r"C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\midi_songs\*.mid"):
    midi = converter.parse(file)
    print("Processing", file)
    parts = instrument.partitionByInstrument(midi)
    if parts:
        music=parts.parts[0].recurse()
    else:
        music=midi.flat.notes

    for element in music:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

with open (r"C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\notes.pkl", "wb") as f:
    pickle.dump(notes, f)

Processing C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\midi_songs\Caught Up In You.mid
Processing C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\midi_songs\Fantasy Girl.mid


In [4]:
with open (r"C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\notes.pkl","rb") as f:
    notes = pickle.load(f)

In [5]:
pitchnames = sorted(set(item for item in notes))
n_vocab = len(pitchnames)

In [6]:
note_to_int = {note:number for number, note in enumerate(pitchnames)}
sequence_length=100

In [7]:
network_input=[]
network_output=[]

In [8]:
for i in range(0, len(notes) - sequence_length):
    seq_in = notes[i:i + sequence_length]
    seq_out = notes[i + sequence_length]

    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

In [9]:
n_patterns=len(network_input)

In [10]:
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)

network_output = np.array(network_output)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation

In [15]:
model=Sequential()

In [17]:
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))

model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
from keras.utils import to_categorical

network_output = to_categorical(network_output)

In [19]:
model.fit(network_input, network_output, epochs=100, batch_size=64)
model.save("music_model.h5")

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 993ms/step - loss: 2.5412
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 996ms/step - loss: 1.9669
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 987ms/step - loss: 1.8630
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 996ms/step - loss: 1.7702
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.8847
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.8106
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.8012
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.8014
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.8302
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - loss: 1.8207
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - loss: 1.8560
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 1.7847
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 1.8150
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - loss: 1.8266
Epoch 15/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 1.7756
Epoch 16/100
6/6 ━━━━━━━━━━━━━━━━━━

In [20]:
import random
from keras.models import load_model

model = load_model("music_model.h5")

In [21]:
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

prediction_output=[]

for _ in range(500):
    prediction_input=np.reshape(pattern, (1, len(pattern), 1))
    prediction = model.predict(prediction_input, verbose=0)

    index=np.argmax(prediction)
    result=pitchnames[index]
    prediction_output.append(result)

    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

In [25]:
from music21 import stream

output_notes=[]

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord=pattern.split('.')
        notes=[]
        for n in notes_in_chord:
            new_note=note.Note(int(n))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord=chord.Chord(notes)
        output_notes.append(new_chord)
    else:
        new_note=note.Note(pattern)
        new_note.storedInstrument=instrument.Piano()
        output_notes.append(new_note)

midi_stream=stream.Stream(output_notes)
midi_stream.write('midi', fp=r'C:\Users\ashis\OneDrive\Documents\Internship\Code Alpha Interndhip\AI Projects\Music Generation\ai_music.mid')

'C:\\Users\\ashis\\OneDrive\\Documents\\Internship\\Code Alpha Interndhip\\AI Projects\\Music Generation\\ai_music.mid'